<a href="https://colab.research.google.com/github/slp22/data-engineering-project/blob/main/engineering_monkeypox_mvp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Data Engineering | MVP

# Monkeypox Tweets

## Imports

In [1]:
import json
import logging
import sqlite3
import matplotlib.pyplot as plt
import numpy as np
import os, shutil, itertools
import pandas as pd
import pathlib as Path
import pickle
import PIL
import random
import seaborn as sns
import sklearn as sk
import warnings
import zipfile

from sqlite3 import connect


## py/spark install



In [52]:
# # https://towardsdatascience.com/pyspark-on-google-colab-101-d31830b238be
# # https://www.analyticsvidhya.com/blog/2020/11/a-must-read-guide-on-how-to-work-with-pyspark-on-google-colab-for-data-scientists/

!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [56]:
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

In [57]:
!tar -xf spark-3.0.0-bin-hadoop3.2.tgz

In [58]:
!pip install -q findspark

In [80]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

In [63]:
import findspark
findspark.init()

In [99]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, explode, col, lower
from pyspark.sql.types import StructType,StructField, StringType, IntegerType


## 1 | Research Design


* **Research Question:** 
* **Impact Hypothesis:** 
* **Data source:** 

* **Data Dictionary:**


## 2 | Data Ingestion

#### [Monkeypox Tweets](https://www.kaggle.com/datasets/aneeshtickoo/tweets-on-monkeypox)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# copy kaggle key from Google Drive
shutil.copyfile('/content/drive/MyDrive/kaggle.json', '/content/kaggle.json')

'/content/kaggle.json'

In [4]:
#confirm kaggle key is in dir
os.listdir()

['.config', 'kaggle.json', 'drive', 'sample_data']

In [5]:
with open('kaggle.json') as json_file:
    kaggle = json.load(json_file)

In [6]:
# assign to directory 
os.environ['KAGGLE_CONFIG_DIR'] = "/content"

In [7]:
# download dataset from kaggle
# 'chmod 600 /content/kaggle.json'
! kaggle datasets download -d aneeshtickoo/tweets-on-monkeypox

  0% 0.00/1.12M [00:00<?, ?B/s]
100% 1.12M/1.12M [00:00<00:00, 111MB/s]


In [8]:
# unzip kaggle file
zip_ref = zipfile.ZipFile('tweets-on-monkeypox.zip', 'r') 
zip_ref.extractall('/content')
zip_ref.close()

## 3 | Exploratory Data Analysis

### First glance

In [85]:
df = pd.read_csv('/content/monkeypox.csv')

In [105]:
df.head(2)

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,1555815462201872385,1555815462201872385,2022-08-06 12:48:06 India Standard Time,2022-08-06,12:48:06,530,820113517613154304,thetenth2022,TheTenth,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
1,1555815458602831872,1555815458602831872,2022-08-06 12:48:05 India Standard Time,2022-08-06,12:48:05,530,196518052,ashemedai,Jeroen Ruigrok van der Werven,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN


In [106]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6859 entries, 0 to 6858
Data columns (total 36 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               6859 non-null   int64  
 1   conversation_id  6859 non-null   int64  
 2   created_at       6859 non-null   object 
 3   date             6859 non-null   object 
 4   time             6859 non-null   object 
 5   timezone         6859 non-null   int64  
 6   user_id          6859 non-null   int64  
 7   username         6859 non-null   object 
 8   name             6859 non-null   object 
 9   place            2 non-null      object 
 10  tweet            6859 non-null   object 
 11  language         6859 non-null   object 
 12  mentions         6859 non-null   object 
 13  urls             6859 non-null   object 
 14  photos           6859 non-null   object 
 15  replies_count    6859 non-null   int64  
 16  retweets_count   6859 non-null   int64  
 17  likes_count   

In [107]:
df.describe()

,id,conversation_id,timezone,user_id,replies_count,retweets_count,likes_count,video,near,geo,source,user_rt_id,user_rt,retweet_id,retweet_date,translate,trans_src,trans_dest
count,6.859000e+03,6.859000e+03,6859.0,6.859000e+03,6859.000000,6859.000000,6859.000000,6859.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,1.555764e+18,1.555192e+18,530.0,7.224884e+17,0.399038,0.691938,3.053944,0.095932,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,2.541498e+13,1.168003e+16,0.0,6.620987e+17,1.789475,11.879167,40.567536,0.294520,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,1.555725e+18,8.510359e+17,530.0,3.989300e+04,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,1.555742e+18,1.555720e+18,530.0,4.906582e+08,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,1.555760e+18,1.555746e+18,530.0,8.977771e+17,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,1.555784e+18,1.555774e+18,530.0,1.395422e+18,0.000000,0.000000,1.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,1.555815e+18,1.555815e+18,530.0,1.555798e+18,68.000000,740.000000,2180.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [108]:
cols_list = list(df.columns)
cols_list

['id',
 'conversation_id',
 'created_at',
 'date',
 'time',
 'timezone',
 'user_id',
 'username',
 'name',
 'place',
 'tweet',
 'language',
 'mentions',
 'urls',
 'photos',
 'replies_count',
 'retweets_count',
 'likes_count',
 'hashtags',
 'cashtags',
 'link',
 'retweet',
 'quote_url',
 'video',
 'thumbnail',
 'near',
 'geo',
 'source',
 'user_rt_id',
 'user_rt',
 'retweet_id',
 'reply_to',
 'retweet_date',
 'translate',
 'trans_src',
 'trans_dest']

In [110]:
df

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,1555815462201872385,1555815462201872385,2022-08-06 12:48:06 India Standard Time,2022-08-06,12:48:06,530,820113517613154304,thetenth2022,TheTenth,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
1,1555815458602831872,1555815458602831872,2022-08-06 12:48:05 India Standard Time,2022-08-06,12:48:05,530,196518052,ashemedai,Jeroen Ruigrok van der Werven,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
2,1555815444359135235,1555815444359135235,2022-08-06 12:48:02 India Standard Time,2022-08-06,12:48:02,530,600467609,democracymotion,Democracy In Motion,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
3,1555815437409157120,1555815437409157120,2022-08-06 12:48:00 India Standard Time,2022-08-06,12:48:00,530,1431886953998258177,thegoogle93,The Google,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
4,1555815424666869761,1555595696639774722,2022-08-06 12:47:57 India Standard Time,2022-08-06,12:47:57,530,1506745578159591428,bufflosouljah1,Bufflosouljah,NaN,...,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'Natrone86', 'name': 'Weed Al...",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6854,1555725304659935232,1555718508813570048,2022-08-06 06:49:51 India Standard Time,2022-08-06,06:49:51,530,1498464506384732162,arkcowgirl62,ValTheCowgirl,NaN,...,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'hurtmeknots', 'name': ""Sambe...",NaN,NaN,NaN,NaN
6855,1555725295432245248,1555725295432245248,2022-08-06 06:49:49 India Standard Time,2022-08-06,06:49:49,530,38338051,speehanagram,Spencer,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
6856,1555725294685900801,1555655517850128384,2022-08-06 06:49:48 India Standard Time,2022-08-06,06:49:48,530,1548821540577464322,pritzkertoilet,Rev. Strychnine Twitch,NaN,...,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'politvidchannel', 'name': 'P...",NaN,NaN,NaN,NaN
6857,1555725290055385089,1555725290055385089,2022-08-06 06:49:47 India Standard Time,2022-08-06,06:49:47,530,298690857,thumbressler,Mike Reux,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN


* RangeIndex: 6859 entries, 0 to 6858
* Data columns (total 36 columns)

### word count

In [115]:
# type(df)
# df.head(2)
pandas_df = df[['date','tweet']]
pandas_df.head(2)

,date,tweet
0,2022-08-06,So has anyone begun to compile 'here's the sta...
1,2022-08-06,"Getting some groceries, topic shifted to covid..."


In [91]:
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [118]:
df_schema = StructType([StructField("date", StringType(), True),
                        StructField("tweet", StringType(), True)])


In [119]:
# tweet_df = spark.read.csv('/content/tweets.csv', inferSchema=True, header='true')

spark_df = spark.createDataFrame(pandas_df, schema=df_schema)
spark_df.printSchema()
spark_df.show()

root
 |-- date: string (nullable = true)
 |-- tweet: string (nullable = true)

+----------+--------------------+
|      date|               tweet|
+----------+--------------------+
|2022-08-06|So has anyone beg...|
|2022-08-06|Getting some groc...|
|2022-08-06|"Illinois Childre...|
|2022-08-06|Illinois daycare ...|
|2022-08-06|@Natrone86 @Junot...|
|2022-08-06|"CDC Recommends L...|
|2022-08-06|1,058,637 America...|
|2022-08-06|MF didn’t know sh...|
|2022-08-06|Three people of A...|
|2022-08-06|MonkeyPox Case: ರ...|
|2022-08-06|#Monkeypox Fallza...|
|2022-08-06|RT-PCR detection ...|
|2022-08-06|@BigBadDenis When...|
|2022-08-06|News Outlook TOP ...|
|2022-08-06|CDC Issues Eye-Op...|
|2022-08-06|United States dec...|
|2022-08-06|@FoxNews They are...|
|2022-08-06|@PetiteNicoco No....|
|2022-08-06|Idc if it’s Monke...|
|2022-08-06|In the technical ...|
+----------+--------------------+
only showing top 20 rows



In [121]:
spark_df = spark_df.withColumn('tweet', 
                               explode(split(lower(col('tweet')), '\s')))

In [122]:
(spark_df.groupBy('tweet')
  .count()
  .orderBy('count', ascending=False)
  .show(10))

+---------+-----+
|    tweet|count|
+---------+-----+
|         | 4898|
|      the| 4351|
|monkeypox| 3230|
|       to| 2957|
|        a| 2400|
|      and| 2344|
|   monkey| 2151|
|       is| 2014|
|       of| 1901|
|        i| 1721|
+---------+-----+
only showing top 10 rows



#❓Count specific words using PySpark?

In [123]:
(spark_df)

DataFrame[date: string, tweet: string]

### Corpus `tweets`

In [124]:
tweets = df[['language','date','username','hashtags','tweet']]
tweets.head(2)

,language,date,username,hashtags,tweet
0,en,2022-08-06,thetenth2022,[],So has anyone begun to compile 'here's the sta...
1,en,2022-08-06,ashemedai,[],"Getting some groceries, topic shifted to covid..."


In [125]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6859 entries, 0 to 6858
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   language  6859 non-null   object
 1   date      6859 non-null   object
 2   username  6859 non-null   object
 3   hashtags  6859 non-null   object
 4   tweet     6859 non-null   object
dtypes: object(5)
memory usage: 268.1+ KB


In [17]:
tweets['username'].nunique()

6028

In [18]:
# How many languages? 
tweets['language'].unique()

array(['en', 'kn', 'de', 'in', 'tl', 'fr', 'pt', 'te', 'tr', 'it', 'qme',
       'bn', 'qht', 'pl', 'es', 'el', 'nl', 'cy', 'ta', 'hi', 'sv', 'ja',
       'th', 'mr', 'et', 'gu', 'da', 'ro', 'ml', 'zxx', 'und', 'pa', 'ur',
       'ko', 'am', 'fi', 'zh', 'lt', 'hu', 'ru', 'ar', 'si'], dtype=object)

In [19]:
# How many tweets in English?
print('English entries:', (tweets[tweets["language"] == 'en'].count())['language'])

# How many tweets in other languages?
print('Spanish entries:', (tweets[tweets["language"] == 'es'].count())['language'])
print('Italian entries:', (tweets[tweets["language"] == 'it'].count())['language'])

English entries: 6410
Spanish entries: 51
Italian entries: 10


In [20]:
# Keep only English languge tweets
tweets = tweets[(tweets['language'] == 'en')]

In [21]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6410 entries, 0 to 6858
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   language  6410 non-null   object
 1   date      6410 non-null   object
 2   username  6410 non-null   object
 3   hashtags  6410 non-null   object
 4   tweet     6410 non-null   object
dtypes: object(5)
memory usage: 300.5+ KB


In [22]:
tweets.head(2)

,language,date,username,hashtags,tweet
0,en,2022-08-06,thetenth2022,[],So has anyone begun to compile 'here's the sta...
1,en,2022-08-06,ashemedai,[],"Getting some groceries, topic shifted to covid..."


In [23]:
# Drop language column since we have only English tweets
tweets = tweets.drop(columns=['language'])

In [24]:
tweets.head(10)

,date,username,hashtags,tweet
0,2022-08-06,thetenth2022,[],So has anyone begun to compile 'here's the sta...
1,2022-08-06,ashemedai,[],"Getting some groceries, topic shifted to covid..."
2,2022-08-06,democracymotion,[],"""Illinois Children's Daycare Worker Tests Posi..."
3,2022-08-06,thegoogle93,[],Illinois daycare worker tests positive for mon...
4,2022-08-06,bufflosouljah1,[],@Natrone86 @JunotIsrael @elcavaqueen @thechicc...
5,2022-08-06,democracymotion,[],"""CDC Recommends Limiting Sex Partners to Avoid..."
6,2022-08-06,salemjakes,[],"1,058,637 Americans have died in the U.S. Covi..."
7,2022-08-06,fineassbrei,[],MF didn’t know shit about monkey pox in June c...
8,2022-08-06,xenohadi,[],Three people of African origin still admitted ...
11,2022-08-06,geopoliticsind,[],RT-PCR detection kit for human monkeypox virus...


In [25]:
# Save corpus
tweets.to_pickle('/content/tweets.pkl')
tweets.to_csv(r'/content/tweets.csv', index=False)

# 4 | Storage

#### SQL Database `monkeypox.db`

##### helper functions

In [27]:
# https://towardsdatascience.com/have-a-sql-interview-coming-up-ace-it-using-google-colab-6d3c0ffb29dc

def pd_to_sqlDB(input_df: pd.DataFrame,
                table_name: str,
                db_name: str = 'default.db') -> None:

    # # Setup local logging
    # logging.basicConfig(level=logging.INFO,
    #                     format='%(asctime)s %(levelname)s: %(message)s',
    #                     datefmt='%Y-%m-%d %H:%M:%S')

    # Find columns in the dataframe
    cols = input_df.columns
    cols_string = ','.join(cols)
    val_wildcard_string = ','.join(['?'] * len(cols))

    # Connect to a DB file if it exists, else create a new file
    con = sqlite3.connect(db_name)
    cur = con.cursor()
    # logging.info(f'SQL DB {db_name} created')

    # Create table
    sql_string = f"""CREATE TABLE {table_name} ({cols_string});"""
    cur.execute(sql_string)
    # logging.info(f'SQL Table {table_name} created with {len(cols)} columns')

    # Upload df
    rows_to_upload = input_df.to_dict(orient='split')['data']
    sql_string = f"""INSERT INTO {table_name} ({cols_string}) VALUES ({val_wildcard_string});"""    
    cur.executemany(sql_string, rows_to_upload)
    # logging.info(f'{len(rows_to_upload)} rows uploaded to {table_name}')
  
    # Commit the changes and close the connection
    con.commit()
    con.close()

In [28]:
#  https://towardsdatascience.com/have-a-sql-interview-coming-up-ace-it-using-google-colab-6d3c0ffb29dc

def sql_query_to_pd(sql_query_string: str, db_name: str ='mpox.db') -> pd.DataFrame:
    
    # Connect to the SQL DB
    con = sqlite3.connect(db_name)

    # Execute the SQL query
    cursor = con.execute(sql_query_string)

    # Fetch the data and column names
    result_data = cursor.fetchall()
    cols = [description[0] for description in cursor.description]

    # Close the connection
    con.close()

    # Return as df
    return pd.DataFrame(result_data, columns=cols)

##### sql_to_df

In [ ]:
# https://towardsdatascience.com/have-a-sql-interview-coming-up-ace-it-using-google-colab-6d3c0ffb29dc

# Read  csv as df
input_df = pd.read_csv('/content/tweets.csv')

# Upload df to a SQL table
pd_to_sqlDB(input_df,
            table_name='tweets',
            db_name='monkeypox.db')

# Write SQL query in a string variable
sql_query_string = """
    SELECT *
    FROM tweets
"""
# Exectue  SQL query
corpus = sql_query_to_pd(sql_query_string, db_name='monkeypox.db')
corpus

* **database = `monkeypox.db`**
* **table_1 = `corpus` (6410 rows × 4 columns)**

In [51]:
# copy corpus csv to Google Drive for Tableau
shutil.copyfile('/content/tweets.csv', '/content/drive/MyDrive/tweets.csv')

'/content/drive/MyDrive/tweets.csv'

# 5 | Processing

* natural language processing, topic modeling (LDA)  
* symptom trends across time + moving average (% or |n|) 

# 6 | Deployment

In [75]:
# streamlit demo: dataframe with countries > adapt to states
# http://localhost:8501/DataFrame_Demo

In [76]:
# dataset with locations to map cases worldwide
# https://www.kaggle.com/datasets/deepcontractor/monkeypox-dataset-daily-updated?select=Monkey_Pox_Cases_Worldwide.csv

# US
# https://www.cdc.gov/poxvirus/monkeypox/response/2022/us-map.html
# https://www.cdc.gov/9cafd3c7-688d-434c-8f1f-7fb329ff618f #csv file

In [ ]:
!pip install streamlit

In [ ]:
!streamlit hello

# 7 | Testing/Robustness

[Python schedule](https://schedule.readthedocs.io/en/stable/examples.html#run-a-job-every-x-minute)